<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<font size="5"><b><center>Стоимость поддержанного автомобиля</center></b></font>

# Описание проекта

## Задача проекта

*Разработать модель для предсказания стоимости продажи автомобиля на вторичном рынке*

## Описание данных

**train.csv** - *информация о характеристиках автомобилей (~440000), которые будут использоваться в качестве обучающих данных.*
 
**test.csv** - *информация о характеристиках автомобилей (~110000), которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'sellingprice' для каждого автомобиля из этого датасета.*
 
**sample_submission.csv** - *пример файла предсказаний в правильном формате.*
 
**vin** - *идентификатор автомобиля в тестовом наборе.*
 
**sellingprice** - *Целевой признак. Для каждого автомобиля предскажите числовое значение стоимости продажи.*

## Описание полей данных

**Year:** *Год выпуска автомобиля.*
  
**Make:** *Марка автомобиля.*

**Model:** *издание автомобиля определенной марки.*

**Trim:** *Уровни отделки салона автомобиля — это просто разные версии модели.*

**Body:** *Тип кузова транспортного средства относится к форме и модели конкретной марки автомобиля.*

**Transmission:** *механизм, который передает мощность от двигателя к колесам.*

**VIN:** *идентификационный номер транспортного средства.*

**State:** *Штат, в котором автомобиль выставлен на аукцион.*

**Condition:** *состояние автомобилей на момент аукциона.*

**Odometer:** *расстояние, пройденное автомобилем с момента выпуска.*

**Color:** *цвет кузова автомобиля.*

**Interior:** *цвет салона автомобиля.*

**Seller:** *продавец автомобиля, автосалоны.*

**mmr:** *рекорд рынка Manhiem, рыночная оценочная цена автомобилей.*

**sellingprice:** *цена, по которой автомобиль был продан на аукционе.*

**saledate:** *дата продажи автомобиля.*

# Библиотеки

In [1]:
import numpy as np
import pandas as pd
import string
from tqdm import tqdm, notebook
from tqdm.notebook import tqdm as log_progress
from vininfo import Vin

from matplotlib.gridspec import GridSpec
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.sparse import csr_matrix

import category_encoders
from sklearn.preprocessing import RobustScaler, OrdinalEncoder

from statsmodels import api as sm

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor, Pool

import warnings
warnings.filterwarnings("ignore")
notebook.tqdm().pandas()

0it [00:00, ?it/s]

# Знакомство с данными

Загрузим данные и посмотрим на их содержимое

In [2]:
train = pd.read_csv("/Users/greygreywolf/Downloads/used-cars-price-prediction-22ds/train.csv")
test = pd.read_csv("/Users/greygreywolf/Downloads/used-cars-price-prediction-22ds/test.csv")

## Тренировочная выборка

Рассмотрим тренировчный набор данных.

In [3]:
train.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


При первичном осмотре видно, что в графе "Тип кузова (body)" параметры могут находится в разных регистрах.             
А в графе "Дата продажи (saledate)" дата указана в разных часовых поясах. Будем учитывать эту информацию при дальнейшей обработке.

Теперь посмотрим на описание тренировочной выборки.

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


Видно, что в данных есть пропуски, так же неверный тип данных. Например, для колонки "Дата продажи (saledate)".

## Тестовая выборка

Рассмотрим тестовый набор данных.

In [5]:
test.tail()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
110053,2011,Jeep,Wrangler,Sport,SUV,automatic,1j4aa2d17bl584330,ca,3.8,66085.0,orange,black,dtg operations inc,Thu Jan 15 2015 03:30:00 GMT-0800 (PST)
110054,2014,Lexus,IS 250,Base,sedan,automatic,jthbf1d23e5007526,fl,4.8,17588.0,gray,black,lexus financial services,Mon Jun 15 2015 11:00:00 GMT-0700 (PDT)
110055,2004,Nissan,Maxima,3.5 SL,Sedan,automatic,1n4ba41e54c831950,va,2.9,124036.0,silver,gray,shirlie slack mitsubishi,Thu Feb 05 2015 01:35:00 GMT-0800 (PST)
110056,2013,Nissan,370Z,Base,coupe,manual,jn1az4eh3dm382431,tx,2.2,27169.0,—,black,nissan motor acceptance corporation,Wed Jun 17 2015 03:30:00 GMT-0700 (PDT)
110057,2012,Chevrolet,Camaro,LS,Coupe,NaN,2g1fa1e32c9193058,pa,3.7,40399.0,blue,black,ingersoll auto of danbury,Fri Jan 09 2015 09:00:00 GMT-0800 (PST)


Ситуация аналагична тренировочной выборке.

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  int64  
 1   make          107997 non-null  object 
 2   model         107979 non-null  object 
 3   trim          107944 non-null  object 
 4   body          107464 non-null  object 
 5   transmission  97047 non-null   object 
 6   vin           110058 non-null  object 
 7   state         110058 non-null  object 
 8   condition     107679 non-null  float64
 9   odometer      110039 non-null  float64
 10  color         109900 non-null  object 
 11  interior      109900 non-null  object 
 12  seller        110058 non-null  object 
 13  saledate      110058 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


В тестовых данных так же есть пропуски и неверный формат данных.

## Вывод

1. В данных есть пропуски.
2. Неверный формат данных.
3. Данные в одной колонке могут быть указаны в разном регистре. Буду учитывать это при дальнейшей обработке.
4. Некоторые составные названия колонок, такие как: "saledate" и "sellingprice", для удобства скорректирую в соответствие со snake_case.

# Предварительная обработка данных

***От автора: Все функции содержат описание. Если необходимо посмотреть описание, и при этом не возращатся к разделу с функциями можно использовать горячие клавиши "shift+tab" (на системе MAC OS/анологичную комбинацию для любой другой OS) выделив функцию.***

## Функции для предобработки

В этом разделе находятся функции для предварительной обработки

In [7]:
def str_to_lower(df):
    '''Функция меняет регистр текста на нижний для колонок с типом "object" переданного датасета.'''
    for column in log_progress(df.columns, leave=True):
        if (df[column].dtype == 'object'):
            df[column] = df[column].str.lower()
    return df

In [8]:
def get_na_for_feature(column):
    """Функция принимает на вход признак и вывод на экран количество пропусков для него в тренировочном и тестовом наборе."""
    print(f'Количество пропусков для признака "{column}" в train -', train[f'{column}'].isna().sum())
    print(f'Количество пропусков для признака "{column}" в test -', test[f'{column}'].isna().sum())

In [9]:
def show_lst_in_str(df, column):
    """Функция примает на вход датасет, признак и вывод на экран строку с уникальными значениями."""
    string = ', '
    lst_wth_uniq = df[column].unique()
    lst_wth_uniq = [str(x) for x in lst_wth_uniq]
    lst_wth_uniq = sorted(lst_wth_uniq)
    string = ', '.join(str(x) for x in lst_wth_uniq)
    print(string)

In [10]:
def get_percent_rat(df, df_name, column):
    """Функция принимает на вход датасет, строковое название датасета и признак.
       И выводит на экран процентное соотношение количества пропусков к общему размеру датасета."""
    perc = (df[column].isna().sum() / df.shape[0]) * 100
    print(f'Количество пропусков в {column} по отношению к общему размеру {df_name} составляет - %.2f' % perc)

In [11]:
def get_dict_wth_mode_for_features(lst_for_group, feature):
    '''Функция принимает на вход список колонок по которым будет происходить группировка,
       признак по которому будет поиск моды. И возвращает словарь, где ключ - это группировка, а значение - мода для
       этой группировки.'''
    dct = None
    df = None
    df = train.groupby(lst_for_group)[feature].agg(lambda x: pd.Series.mode(x,dropna=True)).reset_index()
    df[f'{lst_for_group[0]}_and_{lst_for_group[1]}'] = df[f'{lst_for_group[0]}'] + ',' +  df[f'{lst_for_group[1]}']
    dct = dict(zip(df[f'{lst_for_group[0]}_and_{lst_for_group[1]}'],df[f'{feature}']))     
    return dct

In [12]:
def get_dict_wth_median_for_features(lst_for_group, feature):
    '''Функция принимает на вход список колонок по которым будет происходить группировка,
       признак по которому будет поиск медианы. И возвращает словарь, где ключ - это группировка, а значение - медиана для
       этой группировки.'''
    dct = None
    df = None
    df = train.groupby(lst_for_group)[feature].agg('median').reset_index()
    df[f'{lst_for_group[0]}_and_{lst_for_group[1]}'] = df[f'{lst_for_group[0]}'].astype(str) + ',' + df[f'{lst_for_group[1]}'].astype(object)
    dct = dict(zip(df[f'{lst_for_group[0]}_and_{lst_for_group[1]}'],df[f'{feature}'])) 
    return dct

In [13]:
def get_val_from_dct(row, dct, feature_first, feature_second, feature_target):
    """Функция возвращает значение из словаря.
       На вход принимает: строку датасет, словарь со значениями
                          названия двух признаков в тестовом формате,
                          название признака, для которого нужно извлечь значение, в текстовом формате."""
    val = None
    key_for_dct = f'{str(row[feature_first])},{str(row[feature_second])}'
    if pd.isna(row[feature_target]):
        val = dct.get(key_for_dct, 'other')
        if (type(val) == np.ndarray):
            shape = val.shape
            if shape[0] >= 2:
                val = val[0]
            elif shape[0] == 0:
                val = train[(train['make'] == row['make'])][feature_target].mode()[0]
        elif val == 'other':
            val = train[(train['make'] == row['make'])][feature_target].mode()[0]
    elif ~pd.isna(row[feature_target]):
        val = row[feature_target]
    return val

## Общая предобработка

Скорректирую составные названия колонок в соотвествие со sanke_case для дальнейшего удобства и переведу регистр текстовых колонок в нижний во избежание ошибок.

In [14]:
train.rename(columns = {'saledate':'sale_date', 'sellingprice':'selling_price'}, inplace=True)
test.rename(columns = {'saledate':'sale_date'}, inplace=True )

In [15]:
train = str_to_lower(train)
test = str_to_lower(test)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [16]:
print(' Соотношение общего количество пропусков к размеру датасета -', (sum(train.isna().sum())/ train.shape[0]) * 100)

 Соотношение общего количество пропусков к размеру датасета - 22.0343179567323


Общее количество пропусков достаточно большое по отношению к общему размеру тренировочной выборки, поэтому просто удалять все сразу не буду. Попробую восстановить пропуски в признаках, если их количество значимо по отношению к размеру датасета. Если корректно востановить невозможно или процент пропусков невелик, то буду удалять во избежание искажения данных.

## Марка автомобиля (make)

In [17]:
get_na_for_feature('make')

Количество пропусков для признака "make" в train - 8043
Количество пропусков для признака "make" в test - 2061


В значениях признака присутствует много пропусков. Попробую заполнить их с помощью vin номера. 
Например, с  4-8 знак в vin номере содержится общий портрет автомобиля (модельном ряд, двигатель...)
Для извлечения буду использовать библиотеку vininfo.

In [18]:
train['make'] = train.progress_apply(lambda row: (Vin(row['vin']).manufacturer).lower() if pd.isna(row['make'])\
                                     else row['make'], axis=1)
test['make'] = test.progress_apply(lambda row: (Vin(row['vin']).manufacturer).lower() if pd.isna(row['make'])\
                                     else row['make'], axis=1)

  0%|          | 0/440236 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [19]:
get_na_for_feature('make')

Количество пропусков для признака "make" в train - 0
Количество пропусков для признака "make" в test - 0


Рассмотрим продронее значения признака

In [20]:
show_lst_in_str(train, 'make')

acura, airstream, am, aston martin, audi, bentley, bmw, bmw m, buick, cadillac, cami, chevrolet, chevrolet canada, chevrolet mexico, chevrolet usa, chrysler, chrysler canada, daewoo, daimler ag (sprinter), daimlerchrysler ag/daimler ag, dodge, dodge canada, dodge mexico, dodge tk, dot, ferrari, fiat, fisker, ford, ford truck, genesis, geo, gmc, gmc truck, honda, hummer, hyundai, infiniti, isuzu, jaguar, jeep, kia, lamborghini, land rover, landrover, lexus, lincoln, maserati, mazda, mazda tk, mercedes, mercedes-b, mercedes-benz, mercedes-benz (sprinter), mercury, mini, mitsubishi, nissan, oldsmobile, plymouth, pontiac, porsche, porsche car, porsche suv, ram, rolls-royce, saab, saturn, scion, smart, subaru, suzuki, tesla, toyota, unsupportedbrand, volkswagen, volkswagen commercial vehicles, volvo, vw


In [21]:
show_lst_in_str(test, 'make')

acura, aston martin, audi, bentley, bmw, buick, cadillac, cami, chev truck, chevrolet, chevrolet canada, chevrolet mexico, chevrolet usa, chrysler, chrysler canada, daewoo, daimlerchrysler ag/daimler ag, dodge, dodge canada, dodge mexico, ferrari, fiat, fisker, ford, ford tk, genesis, geo, gmc, gmc truck, honda, hummer, hyundai, hyundai tk, infiniti, isuzu, jaguar, jeep, kia, land rover, landrover, lexus, lexus canada, lincoln, lotus, maserati, mazda, mercedes, mercedes-benz, mercedes-benz (sprinter), mercury, mini, mitsubishi, nissan, oldsmobile, plymouth, pontiac, porsche, porsche car, ram, rolls-royce, saab, saturn, scion, smart, subaru, suzuki, tesla, toyota, volkswagen, volvo, vw


В данных присутствуют неяные дубликаты, например: volkswagen и vw, land rover и  landrover, mazda и  mazda tk (tk обозначает цвет краски) и т.п. Заменим дубликаты.

Кроме того в значениях есть:
 - нераспознанные марки под значением unsupportedbrand. Если таких значений немного, то удалю их.
 - марка airstream. Это компания производитель домов на колесах. Такие данные могут в дальнейшем ухудшить качество модели. Если строк для этой марки не много, то удалю их.
 - марка bmw m и lamborghini, они встречаются только в обучающем наборе. В контесте соревнований, удалю эти марки, что бы не создавать шум в данных.

In [22]:
dct_make = {'am': 'hummer',
            'chevrolet canada': 'chevrolet', 'chevrolet mexico': 'chevrolet', 'chevrolet usa': 'chevrolet', 'chev truck': 'chevrolet',
            'chrysler canada': 'chrysler',
            'daimlerchrysler ag/daimler ag': 'mercedes-benz', 'daimler ag (sprinter)': 'mercedes-benz',
            'dodge tk': 'dodge', 'dodge mexico': 'dodge', 'dodge canada': 'dodge', 'dot': 'dodge',
            'gmc truck': 'gmc',
            'ford tk': 'ford', 'ford truck': 'ford',
            'hyundai tk': 'hyundai',
            'landrover': 'land rover',
            'lexus canada': 'lexus',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes-benz', 'mercedes': 'mercedes-benz', 'mercedes-benz (sprinter)': 'mercedes-benz',
            'porsche car': 'porsche', 'porsche suv': 'porsche',
            'cami': 'toyota',
            'vw': 'volkswagen'}

In [23]:
print('Количество значений с "unsupportedbrand", "airstream", "bmw m", "lamborghini" -', (len(train[train['make'].isin(['unsupportedbrand', 'airstream', "lamborghini", 'bmw m'])] )/ train.shape[0]) * 100)

Количество значений с "unsupportedbrand", "airstream", "bmw m", "lamborghini" - 0.002271508917944012


In [24]:
train['make'] = train['make'].progress_apply(lambda x: dct_make[x] if x in dct_make else x)
test['make'] = test['make'].progress_apply(lambda x: dct_make[x] if x in dct_make else x)

  0%|          | 0/440236 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [25]:
show_lst_in_str(train, 'make')

acura, airstream, aston martin, audi, bentley, bmw, bmw m, buick, cadillac, chevrolet, chrysler, daewoo, dodge, ferrari, fiat, fisker, ford, genesis, geo, gmc, honda, hummer, hyundai, infiniti, isuzu, jaguar, jeep, kia, lamborghini, land rover, lexus, lincoln, maserati, mazda, mercedes-benz, mercury, mini, mitsubishi, nissan, oldsmobile, plymouth, pontiac, porsche, ram, rolls-royce, saab, saturn, scion, smart, subaru, suzuki, tesla, toyota, unsupportedbrand, volkswagen, volkswagen commercial vehicles, volvo


In [26]:
show_lst_in_str(test, 'make')

acura, aston martin, audi, bentley, bmw, buick, cadillac, chevrolet, chrysler, daewoo, dodge, ferrari, fiat, fisker, ford, genesis, geo, gmc, honda, hummer, hyundai, infiniti, isuzu, jaguar, jeep, kia, land rover, lexus, lincoln, lotus, maserati, mazda, mercedes-benz, mercury, mini, mitsubishi, nissan, oldsmobile, plymouth, pontiac, porsche, ram, rolls-royce, saab, saturn, scion, smart, subaru, suzuki, tesla, toyota, volkswagen, volvo


## Издание автомобиля (model)

In [27]:
get_na_for_feature('model')

Количество пропусков для признака "model" в train - 8123
Количество пропусков для признака "model" в test - 2079


### Корректировка значений моделей марок машин

In [28]:
def replace_duplicates_model_for_make(row, dct, make):
    """Функция предназначена для замены дубликатов моделей для указаного бренда.
       На вход принимает строку, словарь типа {'ложное название модели': "истинное название модели"}, бренд.
       Возвращает значение."""
    true_val = 'unknow'
    if (row['model'] in list(dct.keys()))&(row['make'] == make):
        true_val = dct[row['model']]
    else:
        true_val = row['model']
    return true_val

In [29]:
def get_digit_val_of_model(row, make):
    val = row['model'] 
    if (row['make'] == make)&(type(row['model']) == str):
        if (row['model'])[0].isdigit():
            val = row['model'][0:3]         
    return val

#### ACURA

In [30]:
show_lst_in_str(train[train['make'] == 'acura'], 'model')

cl, el, ilx, integra, legend, mdx, nan, rdx, rl, rlx, rsx, tl, tsx, tsx sport wagon, zdx


In [31]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'tsx sport wagon':'tsx',
                                                                              'rdx': 'mdx',
                                                                              'el': 'integra'}, make='acura', axis=1)
test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'tsx sport wagon':'tsx',
                                                                             'rdx': 'mdx',
                                                                             'el': 'integra'}, make='acura', axis=1)

  0%|          | 0/440236 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

#### AUDI

In [32]:
show_lst_in_str(train[train['make'] == 'audi'], 'model')

a3, a4, a5, a6, a7, a8, allroad, allroad quattro, cabriolet, nan, q3, q5, q7, r8, rs 4, rs 5, rs 6, rs 7, s4, s5, s6, s7, s8, sq5, tt, tt rs, tts


In [33]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'rs 7':'rs7', 'rs 5': 'rs5',
                                                                              'rs 6': 'rs6', 'rs 4': 'rs4',
                                                                              'allroad': 'allroad quattro'},
                                      make='audi', axis=1)
test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'rs 7':'rs7', 'rs 5': 'rs5',
                                                                            'rs 6': 'rs6', 'rs 4': 'rs4',
                                                                            'allroad': 'allroad quattro'},
                                    make='audi', axis=1)

  0%|          | 0/440236 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [34]:
train = train[(train['model'] != 'cabriolet')]

#### NISSAN

In [35]:
train[train['model'] == 'x-trail']

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,selling_price,sale_date
13798,2005,nissan,x-trail,le,NaN,NaN,jn8bt08v55w103214,ab,2.0,132822.0,blue,gray,go auto outlet west,4250,thu dec 18 2014 11:30:00 gmt-0800 (pst)


In [36]:
show_lst_in_str(train[train['make'] == 'nissan'], 'model')

200sx, 300zx, 350z, 370z, altima, altima hybrid, armada, cube, frontier, gt-r, juke, leaf, maxima, murano, murano crosscabriolet, nan, nv, nv cargo, nv passenger, nv200, pathfind, pathfinder, quest, rogue, rogue select, sentra, titan, truck, versa, versa note, x-trail, xterra


In [37]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'altima hybrid':'altima',
                                                                              'pathfind': 'pathfinder',
                                                                              'rogue select': 'rogue',
                                                                              'versa note': 'versa',
                                                                              'allroad': 'allroad quattro',
                                                                              'x-trail': 'rogue'},
                                      make='nissan', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'altima hybrid':'altima',
                                                                            'pathfind': 'pathfinder',
                                                                            'rogue select': 'rogue',
                                                                            'versa note': 'versa',
                                                                            'allroad': 'allroad quattro',
                                                                            'x-trail': 'rogue'},
                                      make='nissan', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

#### BMW

In [38]:
show_lst_in_str(train[train['make'] == 'bmw'], 'model')

1 series, 2 series, 3 series, 3 series gran turismo, 320i, 323i, 328i, 4 series, 4 series gran coupe, 5 series, 5 series gran turismo, 6 series, 6 series gran coupe, 7, 7 series, 750i, 750li, 750lxi, 8 series, activehybrid 5, activehybrid 7, activehybrid x6, alp, i8, m, m3, m4, m5, m6, m6 gran coupe, nan, x1, x3, x4, x5, x5 m, x6, x6 m, z3, z4, z4 m


In [39]:
def get_model_from_trim(row, lst_models, make):
    val = None
    if (row['make'] == make) & (row['model'] in lst_models):
        val = (str(row['trim']).split())[0]
    else:
        val = row['model']
    return val

In [40]:
train['model'] = train.progress_apply(get_model_from_trim,
                                      lst_models=['1 series', '2 series', '3 series',
                                                  '3 series gran turismo', '4 series gran coupe', '4 series',
                                                  '5 series', '5 series gran turismo',
                                                  '6 series', '6 series gran coupe',
                                                  '7 series', '8 series',
                                                  'activehybrid', 'activehybrid 5', 'activehybrid 7'],
                                      make='bmw', axis=1)

test['model'] = test.progress_apply(get_model_from_trim,
                                    lst_models=['1 series', '2 series', '3 series',
                                                '3 series gran turismo', '4 series gran coupe', '4 series',
                                                '5 series', '5 series gran turismo',
                                                '6 series', '6 series gran coupe',
                                                '7 series', '8 series',
                                                'activehybrid', 'activehybrid 5', 'activehybrid 7'],
                                     make='bmw', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [41]:
train['model'] = train.progress_apply(get_model_from_trim,
                                      lst_models=['activehybrid'],
                                      make='bmw', axis=1)

test['model'] = test.progress_apply(get_model_from_trim,
                                    lst_models=['activehybrid'],
                                     make='bmw', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [42]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'m6 gran coupe': 'm6',
                                                                              'activehybrid x6': 'x6',
                                                                              'x6 m': 'x6', 'x5 m': 'x5'}, make='bmw',
                                      axis=1)


test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'m6 gran coupe': 'm6',
                                                                            'activehybrid x6': 'x6',
                                                                             'x6 m': 'x6', 'x5 m': 'x5'}, make='bmw',
                                    axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [43]:
train['model'] = train.progress_apply(get_digit_val_of_model, make='bmw', axis=1)
test['model'] = test.progress_apply(get_digit_val_of_model, make='bmw', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [44]:
del get_model_from_trim

#### CADILLAC

In [45]:
show_lst_in_str(train[train['make'] == 'cadillac'], 'model')

ats, catera, cts, cts coupe, cts wagon, cts-v, cts-v coupe, cts-v wagon, deville, dts, eldorado, elr, escalade, escalade esv, escalade ext, escalade hybrid, fleetwood, nan, seville, srx, sts, sts-v, xlr, xts


In [46]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'escalade hybrid': 'escalade',
                                                                              'escalade ext': 'escalade',
                                                                              'cts-v coupe': 'cts-v',
                                                                              'cts-v wagon': 'cts-v',
                                                                              'cts wagon': 'cts',
                                                                              'cts coupe':'cts',
                                                                              'fleetwood': 'ats'},
                                      make='cadillac', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'escalade hybrid': 'escalade',
                                                                             'escalade ext': 'escalade',
                                                                             'cts-v coupe': 'cts-v',
                                                                             'cts-v wagon': 'cts-v',
                                                                             'cts wagon': 'cts',
                                                                             'cts coupe':'cts',
                                                                             'fleetwood': 'ats'},
                                      make='cadillac', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

#### CHEVROLET

In [47]:
show_lst_in_str(train[train['make'] == 'chevrolet'], 'model')

1500, 2500, 3500, astro, astro cargo, avalanche, aveo, black diamond avalanche, blazer, c/k 1500 series, c/k 2500 series, c/k 3500 series, camaro, caprice, capt, captiva sport, cavalier, classic, cobalt, colorado, comm, corsica, corvette, corvette stingray, cruze, equinox, express, express cargo, g1500, g2500, g3500, hhr, impala, impala limited, lumina, malibu, malibu classic, malibu hybrid, malibu maxx, monte carlo, nan, optra, prizm, s-10, s-10 blazer, s10, silverado 1500, silverado 1500 classic, silverado 1500 hybrid, silverado 1500hd, silverado 2500, silverado 2500hd, silverado 2500hd classic, silverado 3500, silverado 3500 classic, silverado 3500hd, sonic, spark, spark ev, ss, ssr, suburban, tahoe, tahoe hybrid, tahoe limited/z71, tracker, trailblazer, trailblazer ext, traverse, uplander, uplandr, venture, volt


In [48]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'silverado 2500hd classic': 'silverado 2500hd',
                                                                              'silverado 3500 classic': 'silverado 3500',
                                                                              'silverado 1500 classic': 'silverado 1500',
                                                                              '1500':'silverado 1500',
                                                                              '2500':'silverado 2500',
                                                                              '3500':'silverado 3500',
                                                                              'g3500':'express',
                                                                              'g1500':'express',
                                                                              'silverado 1500 hybrid':'silverado 1500',
                                                                              'g2500':'express',
                                                                              'capt':'captiva sport',
                                                                              'uplandr':'uplander',
                                                                              'spark ev':'spark',
                                                                              's10':'s-10',
                                                                              's-10 blazer':'s-10',
                                                                              'classic':'caprice',
                                                                              'tahoe limited/z71': 'tahoe',
                                                                              'tahoe hybrid': 'tahoe',
                                                                              'trailblazer ext': 'trailblazer',
                                                                              'astro cargo':'astro',
                                                                              'malibu hybrid':'malibu',
                                                                              'malibu classic':'malibu',
                                                                              'malibu maxx': 'malibu',
                                                                              'express cargo':'express',
                                                                              'corvette stingray':'corvette',
                                                                              'silverado 1500hd':'silverado 1500',
                                                                              'silverado 2500hd':'silverado 2500',
                                                                              'silverado 3500hd':'silverado 3500'},
                                      make='chevrolet', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'silverado 2500hd classic': 'silverado 2500hd',
                                                                            'silverado 3500 classic': 'silverado 3500',
                                                                            'silverado 1500 classic': 'silverado 1500',
                                                                            '1500':'silverado 1500',
                                                                            '2500':'silverado 2500',
                                                                            '3500':'silverado 3500',
                                                                            'g3500':'express',
                                                                            'g1500':'express',
                                                                            'silverado 1500 hybrid':'silverado 1500',
                                                                            'g2500':'express',
                                                                            'capt':'captiva sport',
                                                                            'uplandr':'uplander',
                                                                            'spark ev':'spark',
                                                                            's10':'s-10',
                                                                            's-10 blazer':'s-10',
                                                                            'classic':'caprice',
                                                                            'tahoe limited/z71': 'tahoe',
                                                                            'tahoe hybrid': 'tahoe',
                                                                            'trailblazer ext': 'trailblazer',
                                                                            'astro cargo':'astro',
                                                                            'malibu hybrid':'malibu',
                                                                            'malibu classic':'malibu',
                                                                            'malibu maxx': 'malibu',
                                                                            'express cargo':'express',
                                                                            'corvette stingray':'corvette',
                                                                            'silverado 1500hd':'silverado 1500',
                                                                            'silverado 2500hd':'silverado 2500',
                                                                            'silverado 3500hd':'silverado 3500'},
                                      make='chevrolet', axis=1)

  0%|          | 0/440235 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

In [49]:
train = train[train['model'] != 'comm']

#### CHRYSLER

In [50]:
show_lst_in_str(train[train['make'] == 'chrysler'],'model')

200, 300, 300m, aspen, cirrus, concorde, crossfire, grand, le baron, lhs, nan, pacifica, prowler, pt, pt cruiser, sebring, town, town and country, twn&country, twn/cntry, voyager


In [51]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'300m':'300',
                                                                              'twn&country':'town and country',
                                                                              'town':'town and country',
                                                                              'twn/cntry':'town and country',
                                                                              'pt':'pt cruiser',
                                                                              'le baron':'lebaron'},
                                      make='chrysler', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'300m':'300',
                                                                             'twn&country':'town and country',
                                                                             'town':'town and country',
                                                                             'twn/cntry':'town and country',
                                                                             'pt':'pt cruiser',
                                                                             'le baron':'lebaron'},
                                      make='chrysler', axis=1)

  0%|          | 0/440234 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

#### DODGE

In [52]:
show_lst_in_str(train[train['make'] == 'dodge'],'model')

avenger, b1500, caliber, caravan, challenger, charger, dakota, dart, durango, gr, grand, grand caravan, intrepid, journey, magnum, nan, neon, nitro, ram, ram cargo, ram pickup 1500, ram pickup 2500, ram pickup 3500, ram van, ram3500, spirit, sprinter, sprinter cargo, stratus, viper


In [53]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'gr':'grand caravan',
                                                                              'grand':'grand caravan',
                                                                              'sprinter cargo':'sprinter',
                                                                              'ram cargo':'ram van',
                                                                              'b1500':'ram van',
                                                                              'ram3500':'ram pickup 3500'},
                                      make='dodge', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'gr':'grand caravan',
                                                                            'grand':'grand caravan',
                                                                            'sprinter cargo':'sprinter',
                                                                            'ram cargo':'ram van',
                                                                            'b1500':'ram van',
                                                                            'ram3500':'ram pickup 3500'},
                                     make='dodge', axis=1)

  0%|          | 0/440234 [00:00<?, ?it/s]

  0%|          | 0/110058 [00:00<?, ?it/s]

#### FORD

In [54]:
show_lst_in_str(train[train['make'] == 'ford'],'model')

aspire, c-max energi, c-max hybrid, contour, crown, crown victoria, e, e-150, e-250, e-350, e-series van, e-series wagon, e150, e250, e350, econoline cargo, econoline wagon, edge, escape, escape hybrid, escort, excurs, excursion, expedit, expedition, expedition el, expeditn, explorer, explorer sport, explorer sport trac, f-150, f-150 heritage, f-150 svt lightning, f-250, f-250 super duty, f-350 super duty, f-450 super duty, f150, f250, f350, fiesta, five hundred, flex, focus, focus st, freestar, freestyle, fusion, fusion energi, fusion hybrid, mustang, mustang svt cobra, nan, police, ranger, shelby gt500, taurus, taurus x, tempo, thunderbird, transit connect, transit van, transit wagon, windstar, windstar cargo


In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={'c-max energi':'c-max',
                                                                              'c-max hybrid':'c-max',
                                                                              'e-series van':'e-250',
                                                                              'e-series wagon':'e-350',
                                                                              'explorer sport':'explorer',
                                                                              'fusion energi':'fusion',
                                                                              'fusion hybrid':'fusion',
                                                                              'police':'crown victoria',
                                                                              'fusion hybrid':'fusion',
                                                                              'explorer sport trac':'explorer sport',
                                                                              'f150':'f-150',
                                                                              'expedition el':'expedition',
                                                                              'expeditn':'expedition',
                                                                              'escape hybrid':'escape',
                                                                              'focus st':'focus',
                                                                              'excurs':'excursion',
                                                                              'windstar cargo':'windstar',
                                                                              'f-150 heritage':'f-150',
                                                                              'f-150 svt lightning':'f-150',
                                                                              'f-250':'f-250 super duty',
                                                                              'f-350':'f-350 super duty',
                                                                              'f250':'f-250 super duty',
                                                                              'f350':'f-350 super duty',
                                                                              'expedit':'expedition',
                                                                              'crown':'crown victoria',
                                                                              'taurus x': 'taurus',
                                                                              'e150':'e-150', 'e250':'e-250', 'e350':'e-350',
                                                                              'econoline cargo':'e-250',
                                                                              'econoline wagon':'e-350',
                                                                              'e':'e-350'},
                                      make='ford', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={'c-max energi':'c-max',
                                                                            'c-max hybrid':'c-max',
                                                                            'e-series van':'e-250',
                                                                            'e-series wagon':'e-350',
                                                                            'explorer sport':'explorer',
                                                                            'fusion energi':'fusion',
                                                                            'fusion hybrid':'fusion',
                                                                            'police':'crown victoria',
                                                                            'fusion hybrid':'fusion',
                                                                            'explorer sport trac':'explorer sport',
                                                                            'f150':'f-150',
                                                                            'expedition el':'expedition',
                                                                            'expeditn':'expedition',
                                                                            'escape hybrid':'escape',
                                                                            'focus st':'focus',
                                                                            'excurs':'excursion',
                                                                            'windstar cargo':'windstar',
                                                                            'f-150 heritage':'f-150',
                                                                            'f-150 svt lightning':'f-150',
                                                                            'f-250':'f-250 super duty',
                                                                            'f-350':'f-350 super duty',
                                                                            'f250':'f-250 super duty',
                                                                            'f350':'f-350 super duty',
                                                                            'expedit':'expedition',
                                                                            'crown':'crown victoria',
                                                                            'taurus x': 'taurus',
                                                                            'e150':'e-150', 'e250':'e-250', 'e350':'e-350',
                                                                            'econoline cargo':'e-250',
                                                                            'econoline wagon':'e-350',
                                                                            'e':'e-350'},
                                      make='ford', axis=1)

  0%|          | 0/440234 [00:00<?, ?it/s]

#### GMC

In [ ]:
show_lst_in_str(train[train['make'] == 'gmc'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"1500":"sierra 1500",
                                                                              "2500":"sierra 2500",
                                                                              "envoy xuv":"envoy",
                                                                              "envoy xl":"envoy",
                                                                              "safari cargo": "safari",
                                                                              "savana cargo":"savana",
                                                                              "sierra 1500 classic": "sierra 1500",
                                                                              "sierra 1500 hybrid": "sierra 1500",
                                                                              "sierra 1500hd": "sierra 1500",
                                                                              "sierra 2500hd": "sierra 2500",
                                                                              "sierra 2500hd classic":"sierra 2500",
                                                                              "sierra 3500hd":"sierra 3500",
                                                                              "subrbn":"suburban",
                                                                              "yukon hybrid":"yukon",
                                                                              "yukon xl":"yukon",
                                                                              "yukon denali": "yukon"},
                                      make='gmc', axis=1)


test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"1500":"sierra 1500",
                                                                            "2500":"sierra 2500",
                                                                            "envoy xuv":"envoy",
                                                                            "envoy xl":"envoy",
                                                                            "safari cargo": "safari",
                                                                            "savana cargo":"savana",
                                                                            "sierra 1500 classic": "sierra 1500",
                                                                            "sierra 1500 hybrid": "sierra 1500",
                                                                            "sierra 1500hd": "sierra 1500",
                                                                            "sierra 2500hd": "sierra 2500",
                                                                            "sierra 2500hd classic":"sierra 2500",
                                                                            "sierra 3500hd":"sierra 3500",
                                                                            "subrbn":"suburban",
                                                                            "yukon hybrid":"yukon",
                                                                            "yukon xl":"yukon",
                                                                            "yukon denali": "yukon"},
                                      make='gmc', axis=1)

In [ ]:
train = train[train['model'] != 'siera']

#### HONDA

In [ ]:
show_lst_in_str(train[train['make'] == 'honda'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"crosstour":"accord crosstour",
                                                                              "accord hybrid":"accord",
                                                                              "civic del sol":"civic",
                                                                              "ridgelin":"ridgeline"},
                                      make='honda', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"crosstour":"accord crosstour",
                                                                            "accord hybrid":"accord",
                                                                            "civic del sol":"civic",
                                                                            "ridgelin":"ridgeline"},
                                      make='honda', axis=1)

#### HYUNDAI

In [ ]:
show_lst_in_str(train[train['make'] == 'hyundai'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"santa fe sport":"santa fe",
                                                                              "santa":"santa fe",
                                                                              "genesis coupe":"genesis",
                                                                              "sonata hybrid":"sonata",
                                                                              "elantra coupe":"elantra",
                                                                              "elantra gt": "elantra",
                                                                              "elantra touring":'elantra'},
                                      make='hyundai', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"santa fe sport":"santa fe",
                                                                            "santa":"santa fe",
                                                                            "genesis coupe":"genesis",
                                                                            "sonata hybrid":"sonata",
                                                                            "elantra coupe":"elantra",
                                                                            "elantra gt": "elantra",
                                                                            "elantra touring":'elantra'},
                                      make='hyundai', axis=1)

#### JEEP

In [ ]:
show_lst_in_str(train[train['make'] == 'jeep'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"grand":"grand cherokee",
                                                                              "gr":"grand cherokee",
                                                                              "cherokee":"grand cherokee"},
                                      make='jeep', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"grand":"grand cherokee",
                                                                            "gr":"grand cherokee",
                                                                            "cherokee":"grand cherokee"},
                                      make='jeep', axis=1)

#### LAND ROVER

In [ ]:
show_lst_in_str(train[train['make'] == 'land rover'],'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"rr":"range rover",
                                                                              "rrs":"range rover sport",
                                                                              "range":"range rover",
                                                                              "rangerover": "range rover",
                                                                              "discovery series ii":"discovery"},
                                      make='land rover', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"rr":"range rover",
                                                                            "rrs":"range rover sport",
                                                                            "range":"range rover",
                                                                            "rangerover": "range rover",
                                                                            "discovery series ii":"discovery"},
                                      make='land rover', axis=1)

#### LINCOLN

In [ ]:
show_lst_in_str(train[train['make'] == 'lincoln'],'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"navigator l":"navigator",
                                                                              "town":"town car",
                                                                              "mkz hybrid":"mkz"},
                                      make='lincoln', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"navigator l":"navigator",
                                                                             "town":"town car",
                                                                             "mkz hybrid":"mkz"},
                                      make='lincoln', axis=1)

#### MAZDA

In [ ]:
show_lst_in_str(train[train['make'] == 'mazda'],'model')

In [ ]:
def get_model_for_mazda_from_trim(row):
    val = None
    if (row['make'] == 'mazda') & (row['model'] in ['b-series', 'b-series truck', 'b-series pickup']):
        val = row['trim'][0:6]
    else:
        val = row['model']
    return val

In [ ]:
train['model'] = train.progress_apply(get_model_for_mazda_from_trim, axis=1)
test['model'] = test.progress_apply(get_model_for_mazda_from_trim, axis=1)

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"mazdaspeed mx-5 miata":"mx-5 miata",
                                                                              "3":"mazda3", "6":"mazda3", "rx8":"rx-8",
                                                                              'mazdaspeed 3':'mazda3',
                                                                              'mazdaspeed mazda3':'mazda3',
                                                                              'tribute hybrid':'tribute',
                                                                              'mazdaspeed mazda6':'mazda6',
                                                                              'mazdaspeed protege':'protege'},
                                      make='mazda', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"mazdaspeed mx-5 miata":"mx-5 miata",
                                                                            "3":"mazda3", "6":"mazda3", "rx8":"rx-8",
                                                                            'mazdaspeed 3':'mazda3',
                                                                            'mazdaspeed mazda3':'mazda3',
                                                                            'tribute hybrid':'tribute',
                                                                            'mazdaspeed mazda6':'mazda6',
                                                                            'mazdaspeed protege':'protege'},
                                      make='mazda', axis=1)

In [ ]:
del get_model_for_mazda_from_trim

#### MERCEDES-BENZ

In [ ]:
show_lst_in_str(train[train['make'] == 'mercedes-benz'],'model')

In [ ]:
def get_model_for_mercedes_from_trim(row):
    val = None
    if (row['make'] == 'mercedes-benz') & (row['model'] in ['gl-class', 'e-class', 'sl-class', 'r-class', 'm-class',
                                                            'c-class', 'slk-class', 'cls-class', 's-class',
                                                            'glk-class', 'clk-class', 'g-class', 'cl-class',
                                                            'cla-class', '500-class', 'gla-class', '300-class',
                                                            '420-class', 'b-class electric drive',
                                                            '400-class', '190-class']):
        val = (str(row['trim']).split())[0]
    else:
        val = row['model']
    return val

In [ ]:
train['model'] = train.progress_apply(get_model_for_mercedes_from_trim, axis=1)
test['model'] = test.progress_apply(get_model_for_mercedes_from_trim, axis=1)

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"c230wz":"c230",
                                                                              "c240w":"c240",
                                                                              'c240s':'c240',
                                                                              'c400':'s400'},
                                      make='mercedes-benz', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"c230wz":"c230",
                                                                            "c240w":"c240",
                                                                            'c240s':'c240',
                                                                            'c400':'s400'},
                                    make='mercedes-benz', axis=1)

In [ ]:
del get_model_for_mercedes_from_trim

#### MERCURY

In [ ]:
show_lst_in_str(train[train['make'] == 'mercury'],'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"mountnr":"mountaineer",
                                                                              "grand":"grand marquis",
                                                                              'mariner hybrid':'mariner',
                                                                              'milan hybrid':'milan',
                                                                              'mountnr':'mountaineer'},
                                      make='mercury', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"mountnr":"mountaineer",
                                                                            "grand":"grand marquis",
                                                                            'mariner hybrid':'mariner',
                                                                            'milan hybrid':'milan',
                                                                            'mountnr':'mountaineer'},
                                    make='mercury', axis=1)

#### LEXUS

In [ ]:
show_lst_in_str(train[train['make'] == 'lexus'],'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"gx":"gx 470",
                                                                              "lx":"lx 470"},
                                      make='lexus', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"gx":"gx 470",
                                                                              "lx":"lx 470"},
                                    make='lexus', axis=1)

#### BENTLEY

In [ ]:
show_lst_in_str(train[train['make'] == 'bentley'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"continental gtc speed":"continental gtc",
                                                                              "continental gt speed":"continental gt",
                                                                              'flying spur':'continental flying spur',
                                                                              'continental flying spur speed':'continental flying spur'},
                                      make='bentley', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"continental gtc speed":"continental gtc",
                                                                              "continental gt speed":"continental gt",
                                                                              'flying spur':'continental flying spur',
                                                                              'continental flying spur speed':'continental flying spur'},
                                    make='bentley', axis=1)

#### DAEWOO

In [ ]:
show_lst_in_str(train[train['make'] == 'daewoo'], 'model')

#### PONTIAC

In [ ]:
show_lst_in_str(train[train['make'] == 'pontiac'], 'model')

In [ ]:
train['model'] = train.progress_apply(replace_duplicates_model_for_make, dct={"grand am":"grand",
                                                                              "grand prix": "grand",
                                                                              "montana sv6":"montana",
                                                                              'flying spur':'continental flying spur',
                                                                              'continental flying spur speed':'continental flying spur',
                                                                              'pursuit':'g5'},
                                      make='pontiac', axis=1)

test['model'] = test.progress_apply(replace_duplicates_model_for_make, dct={"grand am":"grand",
                                                                            "grand prix": "grand",
                                                                            "montana sv6":"montana",
                                                                            'flying spur':'continental flying spur',
                                                                            'continental flying spur speed':'continental flying spur',
                                                                            'pursuit':'g5'},
                                    make='pontiac', axis=1)

#### GENESIS

Наблюдение по марке Genesis.

Первоначально Genesis выпускался под маркой Hyundai. Бренд Genesis был официально объявлен независимой маркой 4 ноября 2015 года, выпустив свою первую автономную модель Genesis, Genesis G90, в 2017 году. А значит до 2014 года был не Genesis, а Hyundai Genesis.

In [ ]:
train.loc[(train['year'] <= 2014) & (train['make'] == 'genesis'), 'model'] = 'genesis'
train.loc[(train['year'] <= 2014) & (train['model'] == 'genesis'), 'make'] = 'hyundai'

test.loc[(test['year'] <= 2014) & (test['make'] == 'genesis'), 'model'] = 'genesis'
test.loc[(test['year'] <= 2014) & (test['model'] == 'genesis'), 'make'] = 'hyundai'

### Заполнение пропусков

Для каждой модели авто в каждой стране могло выпускаться несколько модификаций. 
Так , например Subaru выпустила специализированный автомобиль для Национального лыжного патруля на базе 2.5XT turbo. Во много тип модификации зависил от рынка потребителей в каждой стране.

Для заполнение пропусков тестовой выборке введу дополнительный признак "страна производства". Страна производитель зашифрована в номере vin, для извлечения воспользуюсь библиотекой vininfo.
Заполнять буду модой по групперовке страна, марка.

In [ ]:
get_percent_rat(train, 'train', 'model')

In [ ]:
train.dropna(subset=['model'], inplace=True)
train['country'] = train['vin'].progress_apply(lambda x: Vin(x).country)
test['country'] = test['vin'].progress_apply(lambda x: Vin(x).country)

In [ ]:
dct_mode_model = get_dict_wth_mode_for_features(['country', 'make'], 'model')
test['model'] = test.progress_apply(get_val_from_dct, dct=dct_mode_model,
                                    feature_first='country', feature_second='make',
                                    feature_target='model', axis=1)

In [ ]:
show_lst_in_str(test, 'model')

## Уровни отделки салона автомобиля (trim)

In [ ]:
get_na_for_feature('trim')

VIN-код используется для идентификации автомобиля и его характеристик, но не содержит информации о его отделке.

Уровень отделки салона автомобиля зависит от многих факторов, таких как марка и модель автомобиля, комплектация и бюджет владельца.

Некоторые из факторов, которые могут повлиять на уровень отделки салона, включают в себя материалы отделки, качество сборки, наличие дополнительных функций, таких как подогрев сидений, кондиционер, аудиосистема и т.д., а также стоимость автомобиля.

Например, автомобили высокого класса могут иметь более дорогие материалы отделки, такие как натуральная кожа или алькантара, а также более сложные системы безопасности и комфорта. Автомобили среднего класса могут иметь менее дорогие материалы отделки и более простые функции.

In [ ]:
get_percent_rat(train, 'train', 'trim')

In [ ]:
train.dropna(subset=['trim'], inplace=True)
dct_mode_trim = get_dict_wth_mode_for_features(['make', 'model'], 'trim')

In [ ]:
test['trim'] = test.progress_apply(get_val_from_dct, dct=dct_mode_trim,
                                   feature_first='make', feature_second='model',
                                   feature_target='trim', axis=1)

In [ ]:
show_lst_in_str(test, 'trim')

In [ ]:
del dct_mode_trim

## Тип кузова (body)

In [ ]:
get_na_for_feature('body')

In [ ]:
get_percent_rat(train, 'train', 'body')

In [ ]:
train.dropna(subset=['body'], inplace=True)

Тип кузова зависит от производителя и модели автомобиля. Заполню пропуски в тестовом датасете на основе группировки по марке и модели.

In [ ]:
dct_mode_body = get_dict_wth_mode_for_features(['make','model'], 'body')
test['body'] = test.progress_apply(get_val_from_dct, dct=dct_mode_body,
                                   feature_first='make', feature_second='model',
                                   feature_target='body', axis=1)

In [ ]:
get_na_for_feature('body')

In [ ]:
show_lst_in_str(train, 'body')

In [ ]:
show_lst_in_str(test, 'body')

В значениях есть дубликаты, исправлю.

In [ ]:
dct_body = {'regular cab': 'regular-cab', 'cab plus 4': 'cab plus', 'cts coupe': 'coupe',
            'cts wagon': 'wagon', 'cts-v coupe': 'coupe', 'cts-v wagon': 'wagon', 'e-series van': 'van',
            'g convertible': 'convertible', 'g coupe': 'coupe', 'g sedan': 'sedan',
            'g37 convertible': 'convertible', 'g37 coupe': 'coupe', 'genesis coupe': 'coupe',
            'granturismo convertible': 'convertible', 'promaster cargo van': 'van', 'q60 convertible': 'convertible',
            'q60 coupe': 'coupe', 'transit van': 'van', 'tsx sport wagon': 'wagon',
            'ram van': 'van', 'beetle convertible': 'convertible', 'koup':'coupe'}
train['body'] = train['body'].progress_apply(lambda x: dct_body[x] if x in dct_body else x)
test['body'] = test['body'].progress_apply(lambda x: dct_body[x] if x in dct_body else x)

In [ ]:
del dct_mode_body, dct_body

## Коробка передач (transmission)

In [ ]:
get_na_for_feature('transmission')

In [ ]:
get_percent_rat(train, 'train', 'transmission')

В знаке VIN автомобиля, который обозначает трансмиссию, нет конкретного знака или символа. 
Тип трансмиссии зависит от многих факторов, включая модель и год выпуска автомобиля, его назначение.

Заполню пропуски в тесте на основе греппировки по марке и модели.

In [ ]:
dct_mode_tr = get_dict_wth_mode_for_features(['make','model'], 'transmission')
train['transmission'] = train.progress_apply(get_val_from_dct, dct=dct_mode_tr,
                                             feature_first='make', feature_second='model',
                                             feature_target='transmission', axis=1)

test['transmission'] = test.progress_apply(get_val_from_dct, dct=dct_mode_tr,
                                          feature_first='make', feature_second='model',
                                          feature_target='transmission', axis=1)

In [ ]:
show_lst_in_str(train, 'transmission')

In [ ]:
show_lst_in_str(test, 'transmission')

In [ ]:
del dct_mode_tr

## Состояние автомобиля (condition)

Оценка автомобиля салоном зависит от многих факторов, включая состояние автомобиля, его комплектацию, пробег, историю обслуживания и ремонта, а также репутацию марки и модели. 

In [ ]:
get_na_for_feature('condition')

In [ ]:
get_percent_rat(train, 'train', 'condition')

In [ ]:
train.dropna(subset=['condition'], inplace=True)
dct_median_cn = get_dict_wth_median_for_features(['year','model'], 'condition')

In [ ]:
test['condition'] = test.progress_apply(get_val_from_dct, dct=dct_median_cn,
                                        feature_first='year', feature_second='model',
                                        feature_target='condition', axis=1)

Рассмотрим значения признаков

In [ ]:
show_lst_in_str(train, 'condition')

In [ ]:
show_lst_in_str(test, 'condition')

В наборе есть значения c большим количетсвом знаков после запятой, например : 2.0999999999999996 и 2.1. Считаю целесобразными округлить все оценки до целого числа.

In [ ]:
train['condition'] = train['condition'].progress_apply(lambda x: round(x) if type(x)!=str else x)
test['condition'] = test['condition'].progress_apply(lambda x: round(x) if type(x)!=str else x)

In [ ]:
del dct_median_cn

## Пройденное расстояние (odometer)

In [ ]:
get_na_for_feature('odometer')

In [ ]:
get_percent_rat(train, 'train', 'odometer')

In [ ]:
train.dropna(subset=['odometer'], inplace=True)
dct_median_od = get_dict_wth_median_for_features(['condition', 'model'], 'odometer')

In [ ]:
test['odometer'] = test.progress_apply(get_val_from_dct, dct=dct_median_od,
                                       feature_first='condition', feature_second='model',
                                       feature_target='odometer', axis=1)

In [ ]:
show_lst_in_str(test, 'odometer')

В значения присутствуют странные значения для пройденного расстояния. Например: 1.0. Учту и рассмотрю этот факт в исследовательском анализе.

In [ ]:
del dct_median_od

## Цвет кузова (color) и  цвет салона (interior)

In [ ]:
get_na_for_feature('color')

In [ ]:
get_na_for_feature('interior')

In [ ]:
train.dropna(subset=['color', 'interior'], inplace=True)

Оба признака имеют одинаковое количество пропусков. 
Для одной марки и модели в одно и тоже время могли выпускаться машины с разным цветом кузова и комплектации.
Скорее всего цвет салона может входить в комплектацию модели. 

In [ ]:
dct_mode_color = get_dict_wth_mode_for_features(['model', 'body'], 'color')
dct_mode_intr = get_dict_wth_mode_for_features(['model', 'trim'], 'interior')

In [ ]:
test['color'] = test.progress_apply(get_val_from_dct, dct=dct_mode_color,
                                       feature_first='model', feature_second='body',
                                       feature_target='color', axis=1)

test['interior'] = test.progress_apply(get_val_from_dct, dct=dct_mode_intr,
                                       feature_first='model', feature_second='trim',
                                       feature_target='interior', axis=1)

In [ ]:
get_na_for_feature('color')

In [ ]:
get_na_for_feature('interior')

Теперь посмотрим на значения признаков.

In [ ]:
show_lst_in_str(train, 'color')

In [ ]:
show_lst_in_str(test, 'color')

In [ ]:
show_lst_in_str(train, 'interior')

In [ ]:
show_lst_in_str(test, 'interior')

В значениях для этих признаков присутствует прочерк.

В случае для цвета кузова это означает, что цвет кузова может быть любым. Но принимая во внимае, что у нас вторичный рынок автомобилей, то обычно салоны подержанных автомобилей предлагают автомобили только в одном цвете.

Прочерк в графе "Цвет салона" может означать, что салон автомобиля может быть выполнен в различных цветах или что цвет салона не указан.

Заменю прочерк на значение unknow.

In [ ]:
train.loc[train['color'] == '—', 'color'] = 'unknow'
train.loc[train['interior'] == '—', 'interior'] = 'unknow'

test.loc[test['color'] == '—', 'color'] = 'unknow'
test.loc[test['interior'] == '—', 'interior'] = 'unknow'

In [ ]:
del dct_mode_color, dct_mode_intr

## Продавец (seller)

In [ ]:
show_lst_in_str(train, 'seller')

Пропуски отсутстсвуют, но колонка содержит большое количество значений.

In [ ]:
train['seller'].value_counts().to_frame().reset_index().rename(columns={'seller':'counts', 'index':'seller'})

Возможно, тут могут присутствовать неяные дубликаты, но из за специфичных названий при корректировке велика вероятность удалить или исказить данные. На данном этапе оставлю данные для признака в исходном виде.

## Вывод

Проведена предварительная обработка данных, входе которой были удалены  неявные дубликаты, заполнены пропуски и скорректированы значения. В дополнение в значениях некоторых признаков были обнаружены аномалии. В дальнейшем исследую этот факт.

# Исследовательский анализ данных

Посмотрим на распределение значений и их особенности.

## Функции для анализа

Ниже я написал функции для удобства анализа и наглядного графического отображения.

In [ ]:
def get_outliers(df, column):
    '''Функция считает для указанного столбца переданного датасета общее количество выбросов
       и координаты нижней и верхней границы диаграммы размаха.'''
    q_one = df[column].quantile(q=.25)
    q_three  = df[column].quantile(q=.75)
    iqr = q_three - q_one
    left_border = q_one-(1.5*iqr)
    right_border = q_three+(1.5*iqr)
    outliers_left = df[df[column] < left_border]
    outliers_right = df[df[column] > right_border]
    full_outliers = len(outliers_left) + len(outliers_right)
    return(full_outliers, left_border, right_border)

In [ ]:
def plot_charts(df, df_columns):
    '''Функция рисует гистограмму и ящик с усами на одном полотне на переданного датасета,  цифрового признака тренировочного набора. 
       Легенда отображает значение основных показателей: стандартное отклонение (STD), 
       среднее значение (MEAN), медианное значение (MEDIAN), 
       основные показатели так же отображаются на графике.
       На вход может подаваться один или список признаков'''
    num_chrt = len(df_columns)
    for column in df_columns:
        outliers, left_border, right_border = get_outliers(df, column)
        fig = plt.figure(figsize=[15, 8])
        grd = plt.GridSpec(ncols=1,nrows=2, top=0.92, wspace=0.15, hspace=0)
        plt.suptitle(f'Диаграмма размаха и Гистограмма для колонки "{column.upper()}"', fontsize=17)
        fig_ax_1 = fig.add_subplot(grd[1,0])
        plt.grid(True)
        sns.set_style('darkgrid')
        sns.histplot(data=df, x=column, color='green', legend=False)
        mean_line_h = plt.axvline(np.mean(df[column]), 
                                          color='r', 
                                          linestyle='--',
                                          label=f'Среднее значение для признака "{column.upper()}"')
        left_border_line_h = plt.axvline(left_border, 
                                         color='m', 
                                         linestyle=':',
                                         label=f'Нижняя граница размаха для признака "{column.upper()}"')
        right_border_line_h = plt.axvline(right_border, 
                                          color='m', 
                                          linestyle='-.',
                                          label=f'Верхняя граница для признака "{column.upper()}"') 
        median_line_h = plt.axvline(np.median(df[column]), 
                                    color='blue', 
                                    linestyle='-',
                                    label=f'Медианное значение для признака "{column.upper()}"')
        plt.xlabel('Значения признака', fontsize=15)
        plt.ylabel('Количество', fontsize=15)
        fig_ax_2 = fig.add_subplot(grd[0,0])
        sns.set_style('darkgrid')
        plt.grid(True)
        ax = sns.boxplot(x=column, data=df, palette=['green'],
                         orient="h", showcaps=False, notch=True, medianprops={"color": "blue"}, boxprops=dict(alpha=.5))
        mean_line_b = plt.axvline(np.mean(df[column]), 
                                  color='r', 
                                  linestyle='--',
                                  label=f'Среднее значение для признака "{column.upper()}"')
        median_line_b = plt.axvline(np.median(df[column]), 
                                    color='b', 
                                    linestyle='-',
                                    label=f'Медианное значение для признака "{column.upper()}"')
        left_border_line_b = plt.axvline(left_border, 
                                         color='m', 
                                         linestyle=':',
                                         label=f'Нижняя граница размаха для признака "{column.upper()}"')
        right_border_line_b = plt.axvline(right_border, 
                                          color='m', 
                                          linestyle='-.',
                                          label=f'Верхняя граница для признака "{column.upper()}"')
        plt.ylabel(f'"{column.upper()}"', fontsize=15)
        plt.gca().axes.get_xaxis().set_visible(False)       
        plt.legend(title=f'Легенда для признака "{column.upper()}"',
                   handles=[Patch(color='green', alpha=.5, label=f'Признак "{column.upper()}"'),
                            mean_line_h,
                            median_line_h,
                            left_border_line_h,
                            right_border_line_h,
                            Patch(color='none', label=f'Всего выбросов - {outliers:.2f}'),
                            Patch(color='none', label=f'Стандартное отклонение (STD) - {df[column].std():.2f}'),
                            Patch(color='none', label=f'Среднее значение (MEAN) - {np.mean(df[column]):.2f}'),
                            Patch(color='none', label=f'Медианное значение (MEDIAN) - {np.median(df[column]):.2f}'),
                            Patch(color='none', label=f'Нижняя граница размаха для признака "{column.upper()}" - "{left_border:.0f}"'),
                            Patch(color='none', label=f'Верхняя граница для признака "{column.upper()}" - "{right_border:.0f}"'),],
                   edgecolor = 'r',
                   facecolor = 'oldlace',
                   ncol=2,
                   title_fontsize=17,
                   fontsize=15,
                   loc='center',
                   bbox_to_anchor=(0.5, -1.5));

In [ ]:
def plot_bar(df, column_first, column_second, param_for_xtics_rotation='horizontal'):
    """Функция принмает на вход датасет, признаки и рисуеет для них столбчатую диаграмму."""
    plt.figure(figsize=[10, 7])
    plt.title(f'Столбчатая диаграмма для колонки "{column_first.upper()}"', fontsize=17)
    plt.grid(True)
    sns.set_style('darkgrid')
    df = df.sort_values(by=[column_second], ascending=False)
    sns.barplot(data=df, x=column_first, y=column_second)
    plt.xlabel('Значения признака', fontsize=15)
    plt.xticks(rotation=param_for_xtics_rotation)
    plt.ylabel('Количество', fontsize=15)

In [ ]:
def plot_hist(df, column, param_for_xtics_rotation='horizontal'):
    """Функция принмает на вход датасет, текстовый признак и рисуеет для него гистограмму."""
    plt.figure(figsize=[10, 7])
    plt.title(f'Гистограмма для колонки "{column.upper()}"', fontsize=17)
    plt.grid(True)
    sns.set_style('darkgrid')
    sns.histplot(data=df, x=column, color='green', legend=False)
    plt.xlabel('Значения признака', fontsize=15)
    plt.xticks(rotation=param_for_xtics_rotation)
    plt.ylabel('Количество', fontsize=15)

In [ ]:
def plot_line(df, column_first, column_second, param_for_xtics_rotation='horizontal'):
    """Функция принмает на вход датасет, признаки и рисуеет для них диаграмму рассеяния."""
    plt.figure(figsize=[8, 5])
    plt.title(f'Линейная зависимость для колонок "{column_first.upper()} и {column_second.upper()}"', fontsize=17)
    plt.grid(True)
    sns.set_style('darkgrid')
    sns.lineplot(data=df, x=column_first, y=column_second, legend=False)
    plt.xlabel(f'{column_first.upper()}', fontsize=15)
    plt.xticks(rotation=param_for_xtics_rotation)
    plt.ylabel(f'{column_second.upper()}', fontsize=15)    

In [ ]:
def plot_scatter(df, column_first, column_second, for_hue_and_size):
    """Функция принмает на вход датасет, признаки и рисуеет для них график линейной зависимости"""
    plt.figure(figsize=[13, 7])
    plt.title(f'Диаграмма рассения для колонок "{column_first.upper()} и {column_second.upper()}"', fontsize=17)
    plt.grid(True)
    sns.set_style('darkgrid')
    sns.scatterplot(data=df, x=column_first, y=column_second,
                    hue=for_hue_and_size, size=for_hue_and_size,
                    sizes=(20, 200), palette=plt.get_cmap('tab10_r').reversed())
    plt.xlabel(f'{column_first.upper()}', fontsize=15)
    plt.xticks(rotation='vertical')
    plt.ylabel(f'{column_second.upper()}', fontsize=15)  
    plt.legend(title=f'Количественная информация для признака "{for_hue_and_size.upper()}"',
               edgecolor = 'r',
               facecolor = 'oldlace',
               ncol=2,
               title_fontsize=17,
               fontsize=15,
               loc='center',
               bbox_to_anchor=(0.5, -0.38));

Написанная ниже функция повзоляет приблизительно увидеть влияние переданного в нее признака на цену. 
Поэтому сделана попытка создать однородную среду на основе одной самой распростанненой модели машины
с одинаковыми параметрами для наилучшего отображения влияния. 
       
Такой метод не является 100% показателем влияния, так как могут быть другие внешние факторы!
В дополнее к ней в последующих разделах будет сделана проверка корреляции.

In [ ]:
def get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr(selected_feature):
    """
       Функция принимает на вход название признака, как строку.
       На выходе будет датасет, отфильтрованный по переданному признаку и
       содержащий данные для одной наиболее модели распространенной
       с максимально возможными одинаковыми параметрами.
       
       В наборе будет колонка для указанного признака (признак использовался для группировки),
       цена за общее кол-во проданных машин, цена за одну, количетсво проданных машин.
    """
    clmns = ['model', 'age_of_the_car', 'condition', 'transmission', 'body', 'odometer_categories', 'country', 'color']
    if selected_feature in clmns:
        clmns.remove(selected_feature)
    group_by_duplicated_string = train.groupby(clmns, as_index=False).size().sort_values(by='size', ascending=False)
    max_count = group_by_duplicated_string.index[0]
    popul_df_row = group_by_duplicated_string.loc[[max_count]]
    list_with_part_for_query_string = []
    for i in range(len(popul_df_row.columns)-1):
        name_column = popul_df_row.columns.tolist()[i]
        value_for_column = popul_df_row[name_column].values[0]
        if type(value_for_column)==str:
            value_for_column = f"'{value_for_column}'"
        part_query_string = f'({name_column}=={value_for_column})'
        list_with_part_for_query_string.append(part_query_string)
    query_string = '&'.join(p for p in list_with_part_for_query_string)
    clmns.extend([selected_feature, 'selling_price'])
    df_filterd_by_param = train.query(query_string)[clmns]
    group_by_chosen_feature = df_filterd_by_param.groupby(selected_feature)['selling_price'].agg(['sum', 'count']).reset_index().rename(columns={'sum':'sum_price'})
    group_by_chosen_feature['price_for_one'] = group_by_chosen_feature['sum_price'] / group_by_chosen_feature['count']
    return group_by_chosen_feature

## Дата продажи

In [ ]:
train['sale_date'].unique()

В значениях даты продажи присутствуют pdt(gmt-0700) и pst(gmt-0800). PDT(gmt-0700) и PST(gmt-0800) - это названия двух часовых поясов в США. PDT (Pacific Daylight Time) - это часовой пояс, который используется в западной части страны, а PST (Pacific Standard Time) - в восточной части страны. Оба часовых пояса находятся на западном побережье США и используют летнее время. Разница между PDT и PST составляет один час.

Считаю, что от этой части в значения даты продажи можно избавится, ценной информации она не несет.

В дополнение, по выше перечисленным часовым поясам могу сделать вывод, что предоставлены данные по вторичному рынку автомобилей в США.

Выведу в отдельные колонки год, месяц, день и день недели.

In [ ]:
train['sale_date'] = train['sale_date'].apply(lambda x: x[4:24])
train['sale_date'] = pd.to_datetime(train['sale_date'])
train['sale_date'] = train['sale_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
train['sale_date'] = pd.to_datetime(train['sale_date'])
train['year_sale'] = train['sale_date'].dt.year
train['month_sale'] = train['sale_date'].dt.month
train['day_sale'] = train['sale_date'].dt.day
train['dayofweek_sale'] = train['sale_date'].dt.dayofweek

test['sale_date'] =  test['sale_date'].apply(lambda x: x[4:24])
test['sale_date'] = pd.to_datetime(test['sale_date'])
test['sale_date'] = test['sale_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
test['sale_date'] = pd.to_datetime(test['sale_date'])
test['year_sale'] = test['sale_date'].dt.year
test['month_sale'] = test['sale_date'].dt.month
test['day_sale'] = test['sale_date'].dt.day
test['dayofweek_sale'] = test['sale_date'].dt.dayofweek

### Год продажи (year_sale)

In [ ]:
year_sale_and_number_of_sale = train.groupby('year_sale')['make'].agg('count').reset_index().rename(columns={'make': 'number_of_sale'})
plot_bar(year_sale_and_number_of_sale, 'year_sale', 'number_of_sale')                                                                             

Продажи были только в 2014 и 2015 году. И в 2015 году больше всего. Возможно срез данных сделан за два года. Мало вероятно, что в предыдущие года продаж совсем не было, учитывая востребованность авторынка.

### Месяц продажи(month_sale)

In [ ]:
months_sale_and_number_of_sale = train.groupby('month_sale')['make'].agg('count').reset_index().rename(columns={'make':'number_of_sale'})
plot_bar(months_sale_and_number_of_sale, 'month_sale', 'number_of_sale')

Лучшие продажи были в январе-феврале, мае-июне, декабре. Но это данные за два года в совокупности. Посмотрим по каждому году.

Для 2014 года:

In [ ]:
month_sale_and_number_of_sale = train[train['year_sale'] == 2014].groupby('month_sale')['make'].agg('count').reset_index().rename(columns={'make':'number_of_sale'})
plot_bar(month_sale_and_number_of_sale, 'month_sale', 'number_of_sale')

Большие продажи были только в декабре.

Для 2015:

In [ ]:
month_sale_and_number_of_sale = train[train['year_sale'] == 2015].groupby('month_sale')['make'].agg('count').reset_index().rename(columns={'make':'number_of_sale'})
plot_bar(month_sale_and_number_of_sale, 'month_sale', 'number_of_sale')

Крупные продажи были с января по июнь, потом значения идут на спад.

Это может быть связано с тем, что в летние месяцы люди чаще путешествуют на автомобилях и покупают новые автомобили для своих семей. Также возможно, что это связано с сезонными факторами, такими как погода и доступность топлива.

### Число/день продажи (day_sale)

In [ ]:
day_sale_and_number_of_sale = train.groupby('day_sale')['make'].agg('count').reset_index().rename(columns={'make': 'number_of_sale'})
plot_bar(day_sale_and_number_of_sale, 'day_sale', 'number_of_sale')  

Пики покупок приходятся на начало месяца: 1,2,3,6.

Люди могут покупать машины в начале месяца, чтобы избежать высоких цен в конце месяца из-за повышенного спроса.

Повышенный спрос может быть связан с тем, что многие откладывают покупку автомобиля до конца месяца, надеясь получить лучшую цену или специальные предложения. Или конец месяца часто связан с праздниками и выходными днями, когда люди могут больше времени проводить за рулем и использовать свои автомобили.

### День недели продажи (dayofweek_sale)

In [ ]:
dayofweek_sale_and_number_of_sale = train.groupby('dayofweek_sale')['make'].agg('count').reset_index().rename(columns={'make': 'number_of_sale'})
plot_bar(dayofweek_sale_and_number_of_sale, 'dayofweek_sale', 'number_of_sale')  

В середине недели покупали чаще. Возможно, в эти преиоды было больше доступных автомобилей, которые можно купить, или более благоприятная погода для поездок.

#### Возраст машины

Введу еще один признак "возраст машины".

In [ ]:
train['age_of_the_car'] = train['year_sale'] - train['year']
test['age_of_the_car'] = test['year_sale'] - test['year']

In [ ]:
plot_charts(train, ['age_of_the_car'])

В значениях есть аномалии. Среднее больше медианы, есть скос в положительную сторону. То есть машины, у которых возраст больше 15 лет. Думаю, что это вполне возможно. И удалять такие строки не буду.
Значения равные нулю означают, что была продана новая машина. Так же есть значения равные -1.

In [ ]:
train[train['age_of_the_car'] == -1]

Год сборки автомобиля не может быть больше года начала продаж. Но может быть так, что дата сборки меньше даты старта продаж. Машина могла быть оплачена заранее, и покупатель ждал ее доставки, по разным причинам:

    – Производство может занимать больше времени, чем ожидалось, из-за технических проблем или нехватки комплектующих.
    – Производители могут изменять конструкцию автомобиля во время производства, что может привести к задержкам в 
    производстве.
    – Некоторые производители могут использовать различную маркировку года выпуска для своих автомобилей, что может 
    приводить к путанице.
    - Некоторые автомобили могут быть собраны в разных странах в разные годы, что также может привести к разнице в 
    дате выпуска и дате начала продаж.
    


Заменю все значения равные -1 на 0, а год продажи уменьшу на 1.

In [ ]:
train.loc[train['age_of_the_car'] == -1, 'year'] = 2014
train.loc[train['age_of_the_car'] == -1, 'age_of_the_car'] = 0

test.loc[test['age_of_the_car'] == -1, 'year_sale'] = 2014
test.loc[test['age_of_the_car'] == -1, 'age_of_the_car'] = 0

In [ ]:
age_of_the_car_and_median_price = train.groupby('age_of_the_car')['selling_price'].sum().reset_index().rename(columns={'selling_price': 'sum_price'})
plot_line(age_of_the_car_and_median_price, 'age_of_the_car', 'sum_price')

На графике прослеживается зависимость цены от возраста машины. Чем старше, тем ниже цена. Покупатели готовы платить за более свежие машины чаще, чем за старые. Пик в отметке 2 года означет, что покупатели хотят более свежую машину но не по самой высокой цене, как это может быть за новую или годовалую. Видимо, выбирают более лучший вариант по приемлемой цене.

Для одной идентичной модели на графике можно увидеть некоторую зависимость цены от возраста. 

#### Сезонность (season)

Введу еще один признак - сезон.

In [ ]:
def get_season(val):
    dct = {'9':'autum','10':'autum','11':'autum',
           '12':'winter','1':'winter','2':'winter',
           '3':'spring','4':'spring', '5':'spring',
           '6':'summer','7':'summer','8':'summer'}
    season = dct.get(str(val), 'unknow')
    return season

In [ ]:
train['season'] = train['month_sale'].progress_apply(get_season)
test['season'] = test['month_sale'].progress_apply(get_season)

In [ ]:
plot_hist(train, 'season')

Зимой и весной покупали чаще всего.

In [ ]:
del months_sale_and_number_of_sale, year_sale_and_number_of_sale, month_sale_and_number_of_sale, day_sale_and_number_of_sale, dayofweek_sale_and_number_of_sale, age_of_the_car_and_median_price, get_season

#### Количетво владельцев

Предположим, что в среднем машину меняют раз в три года. Попробую предположить, сколько было владельцев у машины.

In [ ]:
train['number_of_owners'] = train['age_of_the_car'].progress_apply(lambda x: x // 3 if (x>=3) else 1)
test['number_of_owners'] = test['age_of_the_car'].progress_apply(lambda x: x // 3 if (x>=3) else 1)

## Год выпуска автомобиля

In [ ]:
plot_charts(train, ['year'])

Стандартное отклонение не велико и меньше среднего, распределение стремится к нормальному, но содержит аномалии . Некоторые значения отличаются от остальных и не соответствуют общей тенденции. 

Среднее меньше медианы. Значит, признак имеет отрицательную скошенность в сторону хвоста с наибольшим количество аномальных значений. Рассмотрим подробнее эти значения.

In [ ]:
train[train['year'] < 1998]['year'].value_counts()

Аномалии отсутстсвуют, год выпуска имеет реальные значения. Многие модели разных производителей насчитывают десятилетние циклы перевыпусков. Так одна модель могла выпускаться втечение длительного времени, но разными поколениями. Считаю, что удалять такие значения не нужно.

In [ ]:
year_and_car = train.groupby('year')['make'].count().reset_index().rename(columns={'make':'number_of_car'})
plot_line(year_and_car, 'year', 'number_of_car')

Кроме того можно заметить, что чем свежее год выпуска, тем больше таких машин выставляют на продажу. Так в период  выпуска автомобилей с 1995 по 2014, количество машин в продаже растет. Основной пик пришелся на 2012 год. Машины выпуска 2012,2013,2014 преобладают в количестве над остальными.

In [ ]:
year_and_sale = train.groupby('year')['sale_date'].agg('count').reset_index().rename(columns={'sale_date': 'number_of_sale'})
plot_line(year_and_sale, 'year', 'number_of_sale')

График линейной зависимости сделанный по годам выпуска авто с подсчетом количества дат продаж также иллюстрирует, что более свежие машины лучше продаются. Так же будет симетричен вывод по общей выручке за машины конкретного года. Чем больше машин продано определенного года выпуска, тем больше выручка для этого года выпуска авто.

In [ ]:
del year_and_car, year_and_sale

## Марка автомобиля (make)

In [ ]:
plot_hist(train, 'make', 'vertical')

Больше всего выставлено на продажу машин марки ford, nissan,chevrolet и toyota. На графике отчетливо видны их пиковые значения.
В меньшенстве в основном люксовые автомобили (ferrari, bentley..).

Посмотрю, какие марки продавались лучше всего.

In [ ]:
make_and_sale = train.groupby('make')['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'})
plot_bar(make_and_sale, 'make', 'number_of_sale', 'vertical')

Лидируют ford, nissan, chevrolet, toyota.

Оценим в какого года выпуска марки продавались лучше всего.

In [ ]:
make_by_year_and_sale = train.groupby(['make', 'year'])['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'})
plot_scatter(make_by_year_and_sale, 'make', 'year', 'number_of_sale')

ford, nissan, chevrolet, toyota с 2011-2014 года выпуска были наиболее востребованы, их продано больше всего.

In [ ]:
del make_by_year_and_sale, make_and_sale

## Издание автомобиля (model)

Моделей очень много, и все не помещаются на графике. Сделаю группировку по марке и модели, и посчитаю для них количество продаж. Я уже знаю лидирующие марки, поэтому попробую посмотреть какие буду значения по продажам и отсечь наиболее худшие.

In [ ]:
make_model_number_of_sale = train.groupby(['make', 'model'])['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'})
plot_charts(make_model_number_of_sale,['number_of_sale'])

Среднее значение количества продаж для модели - 500. Рассмотрю подробнее модели с наибольшим показателем.

In [ ]:
plot_scatter(make_model_number_of_sale.query('(number_of_sale >= 1000)'), 'model', 'make', 'number_of_sale')

На графике видны модели с наибольшим количеством продаж: toyota corolla, ford fushion, ford escape, nissan altima, toyota camri

## Коробка передач (transmission)

In [ ]:
plot_hist(train, 'transmission')

Машин с автоматической системой переключения передач выставлено больше.

In [ ]:
train.groupby('transmission')['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'})

Закономерно, что и продавалось таких машин больше.

In [ ]:
train.groupby('transmission')['selling_price'].median().reset_index().rename(columns={'selling_price': 'median_price'})

По общей картине можно сделать вывод: машины с ручной коробкой передач стоили дешевле.

## Тип кузова (body)

In [ ]:
plot_hist(train, 'body', param_for_xtics_rotation='vertical')

Популярные типы кузовоов:

1. Suv - тип кузова, который сочетает в себе характеристики внедорожника и универсала. Он имеет высокую проходимость и может перевозить большое количество груза.

2. Sedan - это один из типов кузова автомобиля. Он отличается от других типов кузовов тем, что имеет 2 или 4 двери, большой багажник и высокий потолок.

3. Hatchback - отличается от седана тем, что у него есть задняя дверь, которая может открываться вверх или назад, как у багажника. Хэтчбеки обычно имеют более компактный размер, чем седаны, и часто используются для перевозки пассажиров и грузов.

4. Minivan - это еще одна разновидность кузова автомобиля, которая отличается от седанов и хэтчбеков тем, что она имеет больше места для пассажиров и багажа. Минивэны обычно имеют 4 или 5 дверей и могут вместить до 8-9 человек

In [ ]:
plot_bar(train.groupby('body')['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'}),
         'body', 'number_of_sale', 'vertical')

Машины с этим типом кузовов продавались чаще.

## Состояние автомобилей на момент аукциона. (condition)

In [ ]:
plot_hist(train, 'condition')

Аномалий нет. Машин с оценкой 4 больше.

In [ ]:
plot_line(train.groupby('condition')['selling_price'].sum().reset_index().rename(columns={'selling_price': 'sum_price'}),
          'condition', 'sum_price')

На графике показана зависимость цены от состояния машины. Чем выше оценка, тем выше цена. Спад цены в отметке 5 может быть обусловлен малыи количетсвом данных, или машин в очень хорошем состояние. Так же возможно, что машины с идеальными состоянием в цене значительно дороже, чем машина с состоянием "на хорошо". Покупатели могут выбирать золотую середину.

Введу признак отношение возраста к состоянию машины.

In [ ]:
train['age_and_condition'] = train.progress_apply(lambda x: x['age_of_the_car'] / x['condition'], axis=1)
test['age_and_condition'] = test.progress_apply(lambda x: x['age_of_the_car'] / x['condition'], axis=1)

## Цвет кузова (color)

In [ ]:
plot_hist(train, 'color', 'vertical')

Основные цвета black, white, gray, silver

In [ ]:
plot_bar(train.groupby('color')['sale_date'].count().reset_index().rename(columns={'sale_date': 'number_of_sale'}),
         'color', 'number_of_sale', 'vertical')

Машины с цветом кузова: black, white, gray, silver лидируют в продажах. Это могут быть стандартные цвета. А более специфичный цвет, может быть дороже, например, для премиум марок.

Возможно на выбор цвета может влиять климатический пояс штата, для которого берут машину. 

Например, для солнечных регионов могут брать более светлые тона, что бы машина меньше нагревалась.

Либо влияние на цену оказывает технологический фактор самой краски, например, она может обладать дополнительными опциями и быть сложнее в производстве.

В данных присутствует значительная доля машин цвет кузова для которых восстановить корректно нельзя.

## Расстояние, пройденное автомобилем с момента выпуска(odometer)

In [ ]:
plot_charts(train, ['odometer'])

1. Я знаю, что изучаю вторичный рынок США, а значит скорее всего расстояние измеряется в милях.
2. Стандартное отклонение показывает большой разброс данных:

   2.1 Есть значения с маленьким показатем( 1 миля и до 10 миль). 

In [ ]:
plot_hist(train[train['odometer'] < 50], 'odometer')

In [ ]:
train[train['odometer'] < 50]

Это могут быть новые машины, или машины со скрученным пробегом, или долго находящиеся в гараже, или это ошибка при выгрузке. Маленький пробег в совокупности с  еденичной оценкой автосалона, небольшим возрастом и низкой ценой так же могут означать, что мащина побывала в дтп, и выставлена на продажу после.

    2.2 Есть машины с высокими показателями( >200 тысяч миль).

In [ ]:
plot_hist(train[train['odometer'] > 200000], 'odometer')

Для некоторых машин значения близкие к 1 млн

In [ ]:
train[train['odometer']>400000]

Возможно, машину часто использовали. Но если возраст небольшой, а показатели пробега близки к млн, возникают сомнения в корректности заполнения данных или же это может быть техническая неисправность одометра.

Заполнять аномалии медианным значением по группировке марка-модель, год-модель и т.п. считаю некорректным, так как это может вызвать искажение данных.

Принимая во внимания все наблюдения, я решил добавить признак "категориии пробега". Так я избавлюсь от выбросов и большого количества значений.

Так же добавлю дополнительные признак для аномально низкого пробега и аномально высокого пробега.

In [ ]:
list_with_bins_for_odometer_intervals = [0,100,1000,30000,50000,100000,200000,400000,np.inf]

train['odometer_categories'] = pd.cut(train['odometer'], list_with_bins_for_odometer_intervals, 
                                  labels=[1,2,3,4,5,6,7,8], include_lowest=True, right=False).astype(int)

test['odometer_categories'] = pd.cut(test['odometer'], list_with_bins_for_odometer_intervals, 
                                  labels=[1,2,3,4,5,6,7,8], include_lowest=True, right=False).astype(int)

In [ ]:
train['is_strange_odometer'] = train['odometer'].progress_apply(lambda x:1 if ((x <= 10)|(x>700000)) else 0)
test['is_strange_odometer'] = test['odometer'].progress_apply(lambda x:1 if ((x <= 10)|(x>700000)) else 0)

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('odometer_categories'), 'price_for_one', 'odometer_categories', 'count')

Машины с небольшим пробегом имеют более высокие оценки от автосалона и более высокую цену.

Машины с большим пробегом имеют больше низких оценок и меньшую стоимость.

Чем больше пробег, тем ниже цена.

Введу дополнительный признак отношение значения одометра к значниению состояния машины, и отношение значение одометра к возрасту машины.

In [ ]:
train['odometer_and_condition'] = train.progress_apply(lambda x: x['condition'] / x['odometer'], axis=1)
train['odometer_and_age'] = train.progress_apply(lambda x: x['age_of_the_car'] / x['odometer'], axis=1)

test['odometer_and_condition'] = test.progress_apply(lambda x: x['condition'] / x['odometer'], axis=1)
test['odometer_and_age'] = test.progress_apply(lambda x: x['age_of_the_car'] / x['odometer'], axis=1)

In [ ]:
del list_with_bins_for_odometer_intervals

## Цена(price)

In [ ]:
plot_charts(train, ['selling_price'])

Высокое значение стандартного отклонения означает, что в данных есть разброс. Среднее стремится в сторону выбросов.

Рассмотрю значение подробнее:

In [ ]:
plot_charts(train[train['selling_price'] < train['selling_price'].quantile(q=.25)], ['selling_price'])

In [ ]:
train[train['selling_price']<2000]['selling_price'].sort_values(ascending=True)

    1. Значение равное 1 похоже на аномалию, удалю его.

In [ ]:
train = train[train['selling_price'] != 1]

    2. Значения больше 35000.
    

In [ ]:
plot_charts(train[train['selling_price'] > train['selling_price'].quantile(q=.75)], ['selling_price'])

На графике видны аномалии. Рассмотрю подробнее.

In [ ]:
train[train['selling_price']>180000]

Автомобилии люксовых брендов по помему мнению могу иметь высокую цену, цена для ford escape скорее всего аномалия. Возможно была допущена ошибка при заполнении.

Сделаю группировку по маркам для медианной цены по ним, что бы увидеть общую картину.

In [ ]:
plot_hist(train[train['selling_price'] > 50000].groupby(['make','model'])['selling_price'].agg('median').reset_index(),
          'make', 'vertical')

Много марок авто бизнес-класса, люксовых, спортивных и внедорожников (повышенной мощности).

Теперь рассмотрю модели.

In [ ]:
plot_scatter(train[train['selling_price'] > 50000], 'model','make', 'selling_price')

Цена для ford escape является неестественно завышенной. Удалю ее.

In [ ]:
train.drop(train[(train['model'] == 'escape')&(train['selling_price'] > 185000)].index, inplace=True)

## Страна сборки (country)

In [ ]:
plot_hist(train, 'country', 'vertical')

На рынке лидируют в количестве машины со сборкой USA, Canada, Japan.

In [ ]:
plot_bar(train.groupby('country')['selling_price'].agg('sum').reset_index(), 'country', 'selling_price', 'vertical')

Покупатели предпочитают сборки USA, Canada, Japan, Germany6 Mexico.

## Продавец (seller)

In [ ]:
seller_price_and_count = train.groupby('seller')['selling_price'].agg(['sum', 'count']).reset_index().rename(columns={'sum': 'sum_price'})

Из за большого количества наименования салонов, они не помещаются на оси, буду использовать выше написанную функцию "get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr" для оценки влияния по выборке.

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('seller'), 'price_for_one', 'seller', 'count')

Цена для одной модели с одинаковыми параметрами различается для разных салонов.

## Штат, в котором сдали машину (state)

In [ ]:
plot_hist(train, 'state', 'vertical')

Филадельфия, Калифорния, Пенсильвания - лидеры по количеству выставленных на продажу машин.

## Уровень отделки салона(trim)

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('trim'), 'price_for_one', 'trim', 'count')

Для одной конкретной модели с одинаковыми параметрами цена в зависимости от уровня отделки салона различается. Но это не является показателем, учитывая большое количество других факторов.

## Цвет салона(interior) 

In [ ]:
plot_hist(train, 'interior', 'vertical')

Цвета black и gray для салона выше по количеству на рынке.

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('interior'), 'price_for_one', 'interior', 'count')

По графику нельзя сделать вывод о влияние цвета на цену, но возможно есть влияние на спрос. Покупатели предпочитают более практичные цвета.

## Влияние признаков на стоимость автомобиля

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('odometer_categories'), 'price_for_one', 'odometer_categories', 'count')

Чем больше пробег, тем ниже цена. Покупатели предпочитают машины с низким пробегом.

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('age_of_the_car'), 'price_for_one', 'age_of_the_car', 'count')

Чем моложе авто, тем выше цена. Большая часть покупатели делают выбор в пользу более свежих машин, от трех до двух лет.

In [ ]:
plot_scatter(get_price_dffrnc_by_one_mdl_bsd_on_sl_ftr('color'), 'price_for_one', 'color', 'count')

Нельяза сделать однозначный выбор влияние цвета кузова на цену. Но заметна тенденция к более темным тонами.

## Отчет

    1. Предоставлен вторичный рынок автомобилей США
    2. Срез данных предоставлен за 2 года
    3. Больше всего продаж было в 2015 году
    4. Покупают в основм более свежие модели, выпуск с 2012-2015 год.
    5. Основная часть покупок была в первой половине года. Покупатели предпочитали начало месяца и середину недели.
    6. Выбирали больше машин марки ford, nissan, chevrolet, toyota. Модели с повышенной вестимостью для пассажиров и багажа, с автоматической коробкой. Цвет предпочитали темный или ближе к нему.
    7. На цену влияет оценка автосалоном состояния машины, а так же пройденное расстояние, марка и модель, возраст авто.
    8. Прорведенная корректировка данных, удалены аномалии.
    9. Для некоторых признаков необходимо сделать дополнительные исследования влияния на целевой признак.
    10. Сгенерированы новые признаки.

# Удаление не информативных и вредных признаков.

In [ ]:
def plot_point_biserial():
    """Функция рисует точечно-бисериальную корреляцию"""
    correlations_point_biserial = [] 
    train_num = train.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
    for column in train_num.drop('selling_price', axis=1).columns:
        correlations_point_biserial.append(stats.pointbiserialr(train_num[column], train_num['selling_price'])[0])
    df_with_point_biserial_corr = pd.DataFrame({'column': train_num.drop('selling_price', axis=1).columns.tolist(),
                                                'correlation point biserial': correlations_point_biserial}) \
                                                .set_index('column')

    fig, ax = plt.subplots(figsize=(30,17))
    plt.title('Точечно-бисериальная корреляция.', fontsize=20)
    sns.heatmap(df_with_point_biserial_corr,
                annot=True,
                cmap="coolwarm",
                fmt='.2g',
                linewidth=4)
    plt.ylabel('Признаки')
    plt.xlabel('Значения')
    plt.grid(True)

In [ ]:
def plot_pirson_map():
    """Функция рисует тепловую карту для корреляции Пирсона"""
    fig, ax = plt.subplots(figsize=(20,12))
    plt.title('Корреляция Пирсона.', fontsize=20)
    sns.heatmap(train.corr(), annot=True, cmap="coolwarm",  fmt='.2g', linewidth=4)
    plt.grid(True);

    В первую очередь будут удалены признаки:
        - sale_date: основная информация из этой колонки уже извлечена
        - vin: не информативен для будущей модели
        - seller: слишком много значений, в том числе уникальных и не пересикающихся для каждого набора. 
        Кодирование такого количества не только замедлит работу модели, но и создаст шум, который может превести к 
        недообучению. Возможно стоит использовать другой метод извлечения информации из признака.
        - state: Считаю признак не информативным.

In [ ]:
columns_to_drop = ['sale_date', 'vin', 'seller', 'state']
train.drop(columns_to_drop, axis=1, inplace=True)
test.drop(columns_to_drop, axis=1, inplace=True)

Я принял решение, попробовать несколько моделей. Заранее сделаю копию для более сложной модели.

In [ ]:
train_for_cat = train.copy(deep=True)
test_for_cat = test.copy(deep=True)

Посмотрим на корреляцию

In [ ]:
plot_pirson_map()

In [ ]:
plot_point_biserial()

    Удалю:
    - year: из за мультиколлинеарности
    - month_sale: как слабый признак
    - odometer_and_age и number_of_owners: из за мультиколлинеарности 

In [ ]:
columns_to_drop = ['year', 'month_sale', 'odometer_and_age', 'number_of_owners']
train.drop(columns_to_drop, axis=1, inplace=True)
test.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
del columns_to_drop

# Кодирование признаков

В наборе присутствуют признаки разных типов. Данные буду кодировать с помощью прямого  и OrdinalEncoder.

In [ ]:
for column in ['country', 'color', 'body', 'make', 'model', 'transmission', 'season', 'interior', 'trim']:    
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    encoder.fit(train[column].values.reshape(-1,1))
    train[column] = encoder.transform(train[column].values.reshape(-1,1))
    test[column] = encoder.transform(test[column].values.reshape(-1,1))

In [ ]:
del encoder, column

# Масштабизация

In [ ]:
scaler = RobustScaler()
for column in train.drop('selling_price', axis=1).columns.tolist():
    train[column] = scaler.fit_transform(train[column].values.reshape(-1, 1))
    test[column] = scaler.transform(test[column].values.reshape(-1, 1))

In [ ]:
del scaler

# Подготовка целевого признака

In [ ]:
target_train = train['selling_price']
train.drop('selling_price', axis=1, inplace=True)

# LinearRegression

Начну с LinearRegression, посмотрим метрику на всех признаках.

In [ ]:
updated_train = csr_matrix(train.values)

x_train, x_test, y_train, y_test = train_test_split(updated_train, target_train, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
results = {'Модель' : 'LR',
           'Значение MAPE на всех признаках' : pd.Series(mean_absolute_percentage_error(y_test, model.predict(x_test)))}

display(pd.DataFrame(results).style.highlight_min(color='red',axis=0).highlight_max(color='lightblue',axis=0))

Посмотрим на значение признаков для модели

In [ ]:
pd.DataFrame(data={'Признака': train.columns,
                   'Коэффицент': model.coef_.flatten()})

Высокие значения могут говорить о том, что модель переобучается, а слишком маленькие (0,02 и т.п.), что для модели они не важны.

Удалю некоторые признаки и посмотрю, как это повлияло на метрику.

In [ ]:
train.drop(['interior', 'make', 'body', 'model', 'trim', 'is_strange_odometer', 'season', 'transmission', 'color'],axis=1, inplace=True)
updated_train = csr_matrix(train.values)
x_train, x_test, y_train, y_test = train_test_split(updated_train, target_train, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
results['Значение MAPE после удаления части признаков']= pd.Series(mean_absolute_percentage_error(y_test, model.predict(x_test)))
display(pd.DataFrame(results).style.highlight_min(color='red',axis=0).highlight_max(color='lightblue',axis=0))

Метрика ухудшилась, результат слабый. Возможно OrdinalEncoder создал избыточную взаимосвязь.

Теперь попробую более сложную модель, CatBoostRegressor.

In [ ]:
del model, results, train, test

# CatBoostRegressor

In [ ]:
train_for_cat.drop(['country', 'selling_price'], axis=1, inplace=True)
test_for_cat.drop(['country'], axis=1, inplace=True)
cat = ['make','model', 'trim', 'body', 'color', 'interior', 'season', 'is_strange_odometer', 'transmission']

In [ ]:
cat_for_search = CatBoostRegressor(loss_function='RMSE',
                                    eval_metric='MAPE',
                                    verbose=False,
                                    early_stopping_rounds=300,
                                    iterations=5000,
                                    learning_rate=0.05,
                                    random_state=42)

params = {'depth': [5,10], 'l2_leaf_reg': [6,9]}
grid = cat_for_search.grid_search(params, Pool(train_for_cat, target_train, cat_features=cat),
                                  shuffle=True, cv=3, verbose=False,  plot=False)

In [ ]:
best_model = CatBoostRegressor(depth=10,
                               l2_leaf_reg=10,
                               loss_function='RMSE',
                               eval_metric='MAPE',
                               early_stopping_rounds=300,
                               learning_rate=0.05,
                               random_seed=42,
                               verbose=False)

best_model.fit(Pool(train_for_cat, target_train, cat_features=cat))

In [ ]:
best_model.best_score_

Результат заметно улудшился.

# Анализ признаков

In [ ]:
features_for_cat = best_model.get_feature_importance(prettified = True)

In [ ]:
plt.figure(figsize=[15, 7])
sns.set_style('darkgrid')
plt.title('ЗНАЧИМОСТЬ ПРИЗНАКОВ.', fontsize=15)
splot = sns.barplot(x='Feature Id', y='Importances', data=features_for_cat.reset_index())
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Показатель', fontsize=13)
plt.xlabel('Признаки', fontsize=13)
plt.xticks(rotation=90);

Значимую роль играет марка, модель, состояние машины и пробег.

# Итоговая модель

Признаки с коэффицентом ниже 3.6 слабозначимы для модели. Удалим их и посмотрим на качество

In [ ]:
importances_features = features_for_cat.set_index('Feature Id').query('Importances > 3.6')

In [ ]:
best_model = CatBoostRegressor(depth=10,
                               l2_leaf_reg=10,
                               loss_function='RMSE',
                               eval_metric='MAPE',
                               early_stopping_rounds=300,
                               learning_rate=0.05,
                               random_seed=42,
                               verbose=False)

best_model.fit(Pool(train_for_cat[importances_features.index], target_train, cat_features=['make', 'body', 'model', 'trim']))

In [ ]:
best_model.best_score_

Метрика незначительно улучшилась.

In [ ]:
predictions = best_model.predict(test_for_cat[importances_features.index])

In [ ]:
submission = pd.read_csv("/Users/greygreywolf/Downloads/used-cars-price-prediction-22ds/sample_submission.csv")
submission['sellingprice'] = predictions
filename = '/Users/greygreywolf/Downloads/car_price_predict.csv'
submission.to_csv(filename, index=False)

# Итоговый отчет

    - Проведена предобработка данных: заполнены пропуски, удалены дубликаты, скорректирован формат данных.
    - Проведен исследовательский анализ: данные исследованы, выявлены закономерности, удалены аномалии,
    сгенерированы новые признаки.
    - Проведен анализ признаков на мультиколлинеарность
    - Выбрана модель
    - Проведен анализ значимости признаков для модели

# Итоговый вывод

    Покупатели отдают предпочтение:
    - большим машина с хорошей вместимостью
    - в хорошем состоянии, но не отличном
    - с небольшим пробегом
    - не моложе года, что бы не переплачивать, но и не слишком старые

Цена на машину зависит не только от бренда и модели, но и от состояния, возраста и пробега. 
Любое увеличение одно из параметров в "лучшую сторону" сопряжено с ростом цены, тогда снижение показывает обратную тенденцию.
Покупатели стараются найти золотую середину. Возможно поэтому им так важно техническое состояние машины, пусть не идеальное, но в тоже время есть возможность выиграть в цене.